In [1]:
import boto3
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os

import sagemaker
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
#bucket = '20210520-sagemaker-yolo-ddp'
prefix = 'data'

role = sagemaker.get_execution_role()

session = boto3.session.Session()
region = session.region_name
print(f"AWS region:{region}")

AWS region:us-east-1


In [2]:
#image = "yolov5-sagemaker-109-cu100-4"  # Example: mask-rcnn-smdataparallel-sagemaker
#image = "yolov5-sagemaker-109-cu111-4"
image = "yolov5-sagemaker-109-cu100-env"
tag = "pt1.8"  # Example: pt1.8

In [3]:
!docker system prune -a -f --volumes
!docker volume rm "docker volume ls -q -f dangling=true"

Deleted Images:
untagged: 815969174475.dkr.ecr.us-east-1.amazonaws.com/yolov5-sagemaker-109-cu110:pt1.8
untagged: 815969174475.dkr.ecr.us-east-1.amazonaws.com/yolov5-sagemaker-109-cu110@sha256:46d599b06143c349863047a226a524a590a763be158d42cd0a8481835b1e5e5e
untagged: yolov5-sagemaker-109-cu110:latest
deleted: sha256:3681c04f82570d3024e73f274bdbc19fc728a41722827e69591955bbda7005fe
deleted: sha256:8684f920d148873d7397a7b8690e030f1b48189dc4335e1f1fc294c7867ba3bc
deleted: sha256:4cfec12f026383d5c07fcb926a905ef9113556f6f4fb772f85b53c00ab31800a
deleted: sha256:857893d2699f12fd522bc2a78e4e6ec152b346603964e5e462e639b551aed9ae
deleted: sha256:61cc7352984d26132611e4746380b92d5b0ad165017e201c53113ecb61636873
deleted: sha256:fa898df6d1d1fecc64469ff90ff51d0deda8359ceb3c6ba380b9575e28a276ad
deleted: sha256:91a923cbe8fe43ad2a6c083ed8508aafea49f3fea44fba6d9d81d62e84f01a59
deleted: sha256:a7eb461c12dc811b8f94a70280aff05a5ccfe3f3ee31ee231481277a4afc6f28
untagged: 763104351884.dkr.ecr.us-east-1.amazonaws

In [4]:
!pygmentize ./Dockerfile

ARG region

# FROM 763104351884.dkr.ecr.${region}.amazonaws.com/pytorch-training:1.8.1-gpu-py36-cu111-ubuntu18.04
FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.7.1-gpu-py36-cu110-ubuntu18.04

#RUN cd /root && \
#	git clone --recursive https://github.com/tkazusa/yolov5-sagemaker-distributed-data-parallel.git && \
#    chmod -R 775 yolov5-sagemaker-distributed-data-parallel && \
#	cd yolov5-sagemaker-distributed-data-parallel && \
#	pip install -r requirements.txt &&\
#    pip install -U smdebug
    

# Install python dependenies
COPY requirements.txt .
RUN python -m pip install --upgrade pip
RUN pip install --no-cache -r requirements.txt coremltools onnx gsutil notebook
RUN pip install -U smdebug

ENV SAGEMAKER_INSTANCE_TYPE ml.p3.16xlarge


In [5]:
!docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE


In [6]:
%%time
! chmod +x build_and_push.sh; bash build_and_push.sh {region} {image} {tag}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  11.49MB
Step 1/7 : ARG region
Step 2/7 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.7.1-gpu-py36-cu110-ubuntu18.04
1.7.1-gpu-py36-cu110-ubuntu18.04: Pulling from pytorch-training

57c49d0f: Pulling fs layer 
40447d26: Pulling fs layer 
2f862619: Pulling fs layer 
278deddf: Pulling fs layer 
80049843: Pulling fs layer 
556b2329: Pulling fs layer 
a0c97a55: Pulling fs layer 
78bd0b24: Pulling fs layer 
278deddf: Waiting fs layer 
80049843: Waiting fs layer 
556b2329: Waiting fs layer 
a0c97a55: Waiting fs layer 
0befbc6f: Pulling fs layer 
cf9fbe3c: Pulling fs layer 
6c31766d: Waiting fs layer 
d58b12f9: Pulling f

In [19]:
import os
from sagemaker.pytorch import PyTorch
from sagemaker.local import LocalSession

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]

instance_type = "ml.p3.16xlarge"  # Other supported instance type: ml.p3.16xlarge, ml.p4d.24xlarge
instance_count = 1  # You can use 2, 4, 8 etc.
docker_image = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:{tag}"  # YOUR_ECR_IMAGE_BUILT_WITH_ABOVE_DOCKER_FILE
job_name = "pytorch-sm-yolo-ddp17"

# SageMakerの操作で使用するローカル用セッション
local_session = LocalSession()

In [20]:
docker_image

'815969174475.dkr.ecr.us-east-1.amazonaws.com/yolov5-sagemaker-109-cu100-env:pt1.8'

In [21]:
estimator = PyTorch(
    entry_point="train.py",
    role=role,
    volume_size=700,
    source_dir=".",
    image_uri=docker_image,
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.8.1",
    sagemaker_session=sagemaker_session,
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
    debugger_hook_config=False
)

In [22]:
data_uri = "s3://sagemaker-us-east-1-815969174475/coco/"
#data_uri = "s3://sagemaker-us-east-1-815969174475/sagemaker-handson-kazusa/"

data_channels = {"data": data_uri}

estimator.fit(inputs=data_channels, job_name=job_name, wait=False)